In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
users = df[['person']].drop_duplicates().copy()

In [ ]:
#Storage más buscado


In [ ]:
df = df.loc[df['timestamp'] > pd.to_datetime('2018-05-15'), :]

In [ ]:
df_maxs = df.groupby('person')['storage'].value_counts()
df_most_events = df_maxs.rename(columns={'storage':'max'}).reset_index().groupby('person').first()[['storage']].reset_index()
df_most_events.rename(columns={'storage':'storage_mas_eventos'}, inplace=True)

In [ ]:
df_views = df.loc[df['event'] == 'viewed product', :]
df_maxs = df_views.groupby('person')['storage'].value_counts()
df_mostView = df_maxs.rename(columns={'storage':'max'}).reset_index().groupby('person').first()[['storage']].reset_index()

In [ ]:
df_mostView.head(10)

In [ ]:
df_checkout = df.loc[df['event'] == 'checkout', :]
df_maxs = df_checkout.groupby('person')['storage'].value_counts()
df_mostCheckout = df_maxs.rename(columns={'storage':'max'}).reset_index().groupby('person').first()[['storage']].reset_index()

In [ ]:
df_mostCheckout.rename(columns={'storage':'storage_mas_checkout'}, inplace=True)

In [ ]:
# res_final = res_final[['storage']]
# res_final.fillna('Unknown', inplace=True)
storages_info_df = pd.merge(users,df_mostView, left_on = 'person', right_on= 'person', how = 'left').set_index('person')
storages_info_df.rename(columns={'storage':'storage_mas_visto'}, inplace=True)

In [ ]:
storages_info_df = pd.merge(storages_info_df,df_mostCheckout, left_index=True, right_on= 'person', how = 'left').set_index('person')

In [ ]:
storages_info_df = pd.merge(storages_info_df,df_most_events, left_index=True, right_on= 'person', how = 'left').set_index('person')

In [ ]:

mymap = {'Nan':0, '256GB':1, '128GB':2, '64GB':3, '32GB':4 , '16GB':5, '8GB':6, '4GB':7, '512MB':8}
storages_info_df = storages_info_df.applymap(lambda s: mymap.get(s) if s in mymap else s)


In [ ]:
storages_info_df.fillna(0, inplace=True)
storages_info_df['suma_total'] = storages_info_df['storage_mas_visto'] + storages_info_df['storage_mas_checkout'] + storages_info_df['storage_mas_eventos']
storages_info_df.to_csv('../data/feature_dispositivos_storage.csv', sep=',')